In [3]:
import sys
import itertools

In [6]:
# Read all characters from standard input
input_chars = open("input.txt").read()

# Convert each character to integer; non-digit characters are converted to 0
digits = [int(c) if c.isdigit() else 0 for c in input_chars]

# Slice the list into pairs (file, free). If there's an odd number of digits, 'free' is treated as 0
pairs = list(itertools.zip_longest(*[iter(digits)]*2, fillvalue=0))

filesystem = []
for id, (file, free) in enumerate(pairs):
    # Add 'id' repeated 'file' times
    filesystem.extend([id] * file)
    # Add 'None' repeated 'free' times
    filesystem.extend([None] * free)

# Function to find the first occurrence of None in the filesystem
def find_freespace(fs):
    try:
        return fs.index(None)
    except ValueError:
        return -1  # Indicates no free space found

# Reorganize the filesystem by filling free spaces with blocks from the end
freespace = find_freespace(filesystem)
while freespace != -1:
    # Pop the last block from the filesystem
    if not filesystem:
        break  # Exit if filesystem is empty
    block = filesystem.pop()
    
    # Continue popping if the block is None
    while block is None and filesystem:
        block = filesystem.pop()
    
    # If a valid block is found, place it in the first free space
    if block is not None:
        filesystem[freespace] = block
    
    # Find the next free space
    freespace = find_freespace(filesystem)

# Calculate the sum of (value * index) for each block in the filesystem
total = sum((val if val is not None else 0) * idx for idx, val in enumerate(filesystem))

# Pretty-print the result
print(total)

6359213660505


In [7]:
# Build the filesystem list as a list of [value, size] pairs
# For each pair and its index, add [id, file] and [None, free]
filesystem = []
for id, (file, free) in enumerate(pairs):
    filesystem.append([id, file])
    filesystem.append([None, free])

# Initialize the pointer to the end of the filesystem
pointer = len(filesystem)

# Loop to reorganize the filesystem
while True:
    pointer -= 1
    if pointer < 0:
        break  # Exit if pointer goes out of bounds

    file = filesystem[pointer]

    # Skip over free spaces ([None, size])
    while file[0] is None:
        pointer -= 1
        if pointer < 0:
            break
        file = filesystem[pointer]

    if pointer < 0:
        break  # Exit if no more files to process

    if pointer == 0:
        break  # Exit if pointer reaches the start

    # Find the first free space where size >= file's size
    freespace = next(
        (i for i, (val, size) in enumerate(filesystem) if val is None and size >= file[1]),
        None
    )

    if freespace is None:
        continue  # No suitable free space found; proceed to the next file

    if freespace > pointer:
        continue  # Free space is after the current file; skip

    # Perform the swap between file and free space
    free_block = filesystem[freespace]
    new_block = [file]  # Start with the current file

    # Calculate the remaining space after placing the file
    extra = free_block[1] - file[1]
    if extra > 0:
        new_block.append([None, extra])  # Add the remaining free space
        pointer += 1  # Adjust the pointer due to the insertion

    # Replace the free space with the new blocks
    filesystem[freespace:freespace + 1] = new_block

    # Mark the original file position as free with the file's size
    filesystem[pointer] = [None, file[1]]

    # Move the pointer back for the next iteration
    pointer -= 1

# Calculate the final sum based on the filesystem state
index = 0
total = 0
for value, length in filesystem:
    for n in range(length):
        # Multiply the value by its current index and add to the total
        total += (n + index) * (value if value is not None else 0)
    index += length  # Update the index based on the block size

# Output the result
print(total)

6381624803796
